In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time

In [2]:
def TDT(n1,n2,n3,n4,n5,n6):
    b = n1+n3+2*n4; c = n2+n3+2*n5
    if b +c == 0:
        return 0
    else:
        return (b-c)**2/(b+c)

In [3]:
def LS(p,q,r,s,t,u): #local sensitivity
    stat = TDT(p,q,r,s,t,u); v = np.zeros(0)
    M = TDT(p,q,r,s,t,u); m = TDT(p,q,r,s,t,u)
    if p >= 1:
        v = [TDT(p-1,q+1,r,s,t,u), TDT(p-1,q,r+1,s,t,u), TDT(p-1,q,r,s+1,t,u), 
             TDT(p-1,q,r,s,t+1,u), TDT(p-1,q,r,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if q >= 1:
        v = [TDT(p+1,q-1,r,s,t,u), TDT(p,q-1,r+1,s,t,u), TDT(p,q-1,r,s+1,t,u), 
             TDT(p,q-1,r,s,t+1,u), TDT(p,q-1,r,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if r >= 1:
        v = [TDT(p+1,q,r-1,s,t,u), TDT(p,q+1,r-1,s,t,u), TDT(p,q,r-1,s+1,t,u), 
             TDT(p,q,r-1,s,t+1,u), TDT(p,q,r-1,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if s >= 1:
        v = [TDT(p+1,q,r,s-1,t,u), TDT(p,q+1,r,s-1,t,u), TDT(p,q,r+1,s-1,t,u), 
             TDT(p,q,r,s-1,t+1,u), TDT(p,q,r,s-1,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if t >= 1:
        v = [TDT(p+1,q,r,s,t-1,u), TDT(p,q+1,r,s,t-1,u), TDT(p,q,r+1,s,t-1,u), 
             TDT(p,q,r,s+1,t-1,u), TDT(p,q,r,s,t-1,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if u >= 1:
        v = [TDT(p+1,q,r,s,t,u-1), TDT(p,q+1,r,s,t,u-1), TDT(p,q,r+1,s,t,u-1), 
             TDT(p,q,r,s+1,t,u-1), TDT(p,q,r,s,t+1,u-1)]
        M = max([max(v),M])
        m = min([min(v),m])
    
    return max([M-stat, stat-m])

In [4]:
def SS(table,beta): #smooth sensitivity
    p = table[0]; q = table[1]; r = table[2]; s = table[3]; t = table[4]; u = table[5]
    GS = 8; ls = LS(p,q,r,s,t,u)
    if beta >= math.log(40/13):
        return ls
    else:
        dist = int(math.ceil(math.log(GS/ls))/beta)
        ss = 0
        for yp in range(-dist,dist+1):
            for yq in range(-dist,dist+1):
                for yr in range(-dist,dist+1):
                    for ys in range(-dist,dist+1):
                        for yt in range(-dist,dist+1):
                            yu = -yp-yq-yr-ys-yt
                            if p+yp >= 0 and q+yq >= 0 and r+yr >= 0 and s+ys >= 0 and t+yt >= 0 and u+yu >= 0:
                                dxy = (math.fabs(yp) + math.fabs(yq) + math.fabs(yr) + math.fabs(ys) + math.fabs(yt) + math.fabs(yu))/2
                                if dxy <= dist:
                                    ss = max([LS(p+yp,q+yq,r+yr,s+ys,t+yt,u+yu)*math.exp(-(beta*dxy)), ss])
        return ss

In [5]:
def generateData(n):
    table = np.zeros(6)
    table[0] = np.random.binomial(n,1/6)
    table[1] = np.random.binomial(n-table[0],1/5)
    table[2] = np.random.binomial(n-table[0]-table[1],1/4)
    table[3] = np.random.binomial(n-table[0]-table[1]-table[2],1/3)
    table[4] = np.random.binomial(n-table[0]-table[1]-table[2]-table[3],1/2)
    table[5] = n-table[0]-table[1]-table[2]-table[3]-table[4]
    return table

In [6]:
def RunTime(N,beta):
    t = 0
    for i in range(100):
        table = generateData(N)
        s = time.time()
        SS(table,beta)
        e = time.time()
        t += (e-s)
    return t/100

In [7]:
N = 1000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

14.216528306007385
0.5188542032241821
3.660440444946289e-05
3.6301612854003904e-05


In [8]:
N = 2000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

21.09646413087845
0.642405366897583
3.482341766357422e-05
3.482580184936523e-05


In [9]:
N = 5000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

28.927685837745667
1.2068973350524903
3.3807754516601565e-05
3.45301628112793e-05
